In [18]:
import language_check
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

/u/jm8wx/.conda/envs/torch/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
data_path = '/p/qdata/jm8wx/research/text_attacks/acl_2020_results/case_study_examples.csv'
df = pd.read_csv(data_path)

In [4]:
df.head()

,USE_similarity,dataset,original_class,original_text,perturbed_class,perturbed_text,run_type
0,0.930254,movie_review,0,davis is so enamored of her own creation that ...,1,davis is even enamored of her own creation tha...,jin
1,0.808168,movie_review,0,"somewhere in the middle, the film compels, as ...",1,"somewhere in the eastwards, the film commits, ...",jin
2,0.930277,movie_review,1,"a muckraking job, the cinematic equivalent of ...",0,"a muckraking job, the cinematic equivalent of ...",jin
3,0.951098,movie_review,0,it's hard to understand why anyone in his righ...,1,it's intense to understand why anyone in his r...,jin
4,0.912998,movie_review,0,never having seen the first two films in the s...,1,never having seen the first two cinema in the ...,jin


In [5]:
lang_tool = language_check.LanguageTool("en-US")

In [20]:
def get_errors(text):
    return lang_tool.check(text)

def get_error_count(text):
    return len(get_errors(text))

In [19]:
df['original_text_error_count'] = df['original_text'].progress_map(get_error_count)
df['perturbed_text_error_count'] = df['perturbed_text'].progress_map(get_error_count)

100%|██████████| 3115/3115 [03:54<00:00, 13.27it/s]


In [21]:
df['original_text_errors'] = df['original_text'].progress_map(get_errors)
df['perturbed_text_errors'] = df['perturbed_text'].progress_map(get_errors)

100%|██████████| 3115/3115 [03:56<00:00, 13.20it/s]


In [30]:
original_text_errors = [e for l in df['original_text_errors'] for e in l]
perturbed_text_errors = [e for l in df['perturbed_text_errors'] for e in l]
print('Original errors:', len(original_text_errors))
print('Perturbed errors:', len(perturbed_text_errors))

Original errors: 49146
Perturbed errors: 52123


In [38]:
print('Added errors:', (df['original_text_error_count'] < df['perturbed_text_error_count']).sum())
print('Same # errors:', (df['original_text_error_count'] == df['perturbed_text_error_count']).sum())
print('Removed errors:', (df['original_text_error_count'] > df['perturbed_text_error_count']).sum())d

Added errors: 1418
Same # errors: 1500
Removed errors: 197


In [47]:
df['run'] = df['dataset'] + '_' + df['run_type']
print(df['run'].unique())

['movie_review_jin' 'yelp_polarity_jin' 'imdb_jin' 'imdb_alzantot']


In [79]:
original_text_error_rule_ids = pd.Series([x.ruleId for x in original_text_errors])
perturbed_text_error_rule_ids = pd.Series([x.ruleId for x in perturbed_text_errors])

original_text_error_rule_freqs = original_text_error_rule_ids.value_counts().to_dict()
perturbed_text_error_rule_freqs = perturbed_text_error_rule_ids.value_counts().to_dict()

print('Original errors:', original_text_error_rule_ids.value_counts())
print()
print('Perturbed errors:', perturbed_text_error_rule_ids.value_counts())

Original errors: MORFOLOGIK_RULE_EN_US            22318
UPPERCASE_SENTENCE_START         11008
I_LOWERCASE                       7744
COMMA_PARENTHESIS_WHITESPACE      3399
EN_QUOTES                         1818
                                 ...  
ENTIRELY_COMPRISED_OF                1
REPEAT_AGAIN                         1
VBZ_VBD                              1
SMALL_NUMBER_OF                      1
NEEDNT_TO_DO_AND_DONT_NEED_DO        1
Length: 231, dtype: int64

Perturbed errors: MORFOLOGIK_RULE_EN_US           23633
UPPERCASE_SENTENCE_START        11016
I_LOWERCASE                      7318
COMMA_PARENTHESIS_WHITESPACE     3400
EN_QUOTES                        1818
                                ...  
OVER_RATED                          1
PERSONAL_OPINION_FRIENDSHIP         1
SCENE_SEEN                          1
AH_HA                               1
PER_SE                              1
Length: 245, dtype: int64


In [113]:
err_data = []

for rule_id, pert_freq in perturbed_text_error_rule_freqs.items():
    orig_freq = original_text_error_rule_freqs.get(rule_id, 0)
    freq_diff = pert_freq - orig_freq
    freq_ratio = pert_freq / orig_freq if orig_freq else None
    err_data.append({
        'freq_diff':  freq_diff,
        'orig_freq':  orig_freq,
        'pert_freq':  pert_freq,
        'freq_ratio': freq_ratio,
        'rule_id': rule_id
    })
    
err_data = pd.DataFrame(err_data)
err_data['freq_diff_perc'] = err_data['freq_diff'] / float(len(df))
err_data.head()

,freq_diff,orig_freq,pert_freq,freq_ratio,rule_id,freq_diff_perc
0,1315,22318,23633,1.058921,MORFOLOGIK_RULE_EN_US,0.422151
1,8,11008,11016,1.000727,UPPERCASE_SENTENCE_START,0.002568
2,-426,7744,7318,0.944990,I_LOWERCASE,-0.136758
3,1,3399,3400,1.000294,COMMA_PARENTHESIS_WHITESPACE,0.000321
4,0,1818,1818,1.000000,EN_QUOTES,0.000000


In [116]:
len(df)

3115

In [114]:
pd.set_option('display.max_rows', None)

err_data.sort_values(by='freq_diff', ascending=False)

,freq_diff,orig_freq,pert_freq,freq_ratio,rule_id,freq_diff_perc
0,1315,22318,23633,1.058921,MORFOLOGIK_RULE_EN_US,0.422151
5,445,110,555,5.045455,EN_A_VS_AN,0.142857
7,306,24,330,13.750000,A_PLURAL,0.098234
8,305,21,326,15.523810,DID_BASEFORM,0.097913
10,186,13,199,15.307692,NON3PRS_VERB,0.059711
14,116,0,116,NaN,I_AM,0.037239
12,103,26,129,4.961538,THIS_NNS,0.033066
11,71,86,157,1.825581,POSSESSIVE_APOSTROPHE,0.022793
17,63,35,98,2.800000,BEEN_PART_AGREEMENT,0.020225
27,44,4,48,12.000000,TO_NON_BASE,0.014125


In [120]:
err_data[err_data['freq_ratio'].isna()].sort_values(by='freq_diff', ascending=False)

,freq_diff,orig_freq,pert_freq,freq_ratio,rule_id,freq_diff_perc
14,116,0,116,NaN,I_AM,0.037239
55,13,0,13,NaN,DONT_NEEDS,0.004173
98,5,0,5,NaN,WAS_BEEN,0.001605
122,3,0,3,NaN,MAN_MEN,0.000963
132,3,0,3,NaN,IS_EVEN_WORST,0.000963
138,2,0,2,NaN,A_BUT,0.000642
148,2,0,2,NaN,SUMMER_TIME,0.000642
160,2,0,2,NaN,NOT_NOTHING,0.000642
169,2,0,2,NaN,MUST_HAVE_TO,0.000642
171,2,0,2,NaN,COMMA_THAN,0.000642


In [122]:
err_data.sort_values(by=['pert_freq'], ascending=False)

,freq_diff,orig_freq,pert_freq,freq_ratio,rule_id,freq_diff_perc
0,1315,22318,23633,1.058921,MORFOLOGIK_RULE_EN_US,0.422151
1,8,11008,11016,1.000727,UPPERCASE_SENTENCE_START,0.002568
2,-426,7744,7318,0.944990,I_LOWERCASE,-0.136758
3,1,3399,3400,1.000294,COMMA_PARENTHESIS_WHITESPACE,0.000321
4,0,1818,1818,1.000000,EN_QUOTES,0.000000
5,445,110,555,5.045455,EN_A_VS_AN,0.142857
6,2,447,449,1.004474,EN_UNPAIRED_BRACKETS,0.000642
7,306,24,330,13.750000,A_PLURAL,0.098234
8,305,21,326,15.523810,DID_BASEFORM,0.097913
9,0,261,261,1.000000,ENGLISH_WORD_REPEAT_RULE,0.000000


In [153]:
len(df)

3115

In [121]:
err_data.sort_values(by=['freq_ratio', 'freq_diff'], ascending=False)

,freq_diff,orig_freq,pert_freq,freq_ratio,rule_id,freq_diff_perc
30,36,2,38,19.000000,AGREEMENT_SENT_START,0.011557
8,305,21,326,15.523810,DID_BASEFORM,0.097913
10,186,13,199,15.307692,NON3PRS_VERB,0.059711
7,306,24,330,13.750000,A_PLURAL,0.098234
27,44,4,48,12.000000,TO_NON_BASE,0.014125
40,21,2,23,11.500000,PRP_PAST_PART,0.006742
64,10,1,11,11.000000,PRP_RB_NO_VB,0.003210
65,10,1,11,11.000000,COULDVE_IRREGULAR_VERB,0.003210
5,445,110,555,5.045455,EN_A_VS_AN,0.142857
89,4,1,5,5.000000,NEEDS_FIXED,0.001284


In [157]:
for err in perturbed_text_errors:
    if err.ruleId == 'MORFOLOGIK_RULE_EN_US':
        badword = err.context[err.contextoffset:err.contextoffset+err.errorlength]
        goodwords = err.replacements
        print(err.msg + ':', 'Replace',badword,'with one of',goodwords)
        print(err.category)
        print()
        break

Possible spelling mistake found: Replace davis with one of ['Davis']
Possible Typo



In [208]:
def yield_errs(err_code, num_to_print=1):
    n = 0
    for err in perturbed_text_errors:
        if n >= num_to_print: 
            break
        if err.ruleId == err_code: 
            badword = err.context[err.contextoffset:err.contextoffset+err.errorlength]
            goodwords = err.replacements
            err_str = err_code + ': ' + err.msg + ' || Replace '+badword+' with one of ' + '[' +','.join(goodwords) + ']'
            context = err.context
            cat = err.category
            yield err_str
            n += 1

def get_err(err_code):
    ans = list(yield_errs(err_code, num_to_print=1))
    return ans[0] if len(ans) else None

def get_errs(*args):
    return list(yield_errs(*args))

def print_errs(*args):
    print(get_errs(*args))

print_errs('I_AM')

["I_AM: Did you mean 'I am'? || Replace i with one of [I am]"]


In [228]:
def yield_errs_context(err_code, num_to_print=1):
    n = 0
    for err in perturbed_text_errors:
        if n >= num_to_print: 
            break
        if err.ruleId == err_code: 
            badword = err.context[err.contextoffset:err.contextoffset+err.errorlength]
            goodwords = err.replacements
            err_str = err_code + ': ' + err.msg + ' || Replace '+badword+' with one of ' + '[' +','.join(goodwords) + ']'
            context = err.context
            cat = err.category
            yield context
            n += 1

def get_err_context(err_code):
    ans = list(yield_errs_context(err_code, num_to_print=1))
    return ans[0] if len(ans) else None

def get_errs_context(*args):
    return list(yield_errs_context(*args))

def print_errs_context(*args):
    print(get_errs_context(*args))

print_errs('I_AM')

["I_AM: Did you mean 'I am'? || Replace i with one of [I am]"]


In [210]:
get_err('DID_BASEFORM')

"DID_BASEFORM: The verb 'can't' requires base form of this verb: 'compare' || Replace compares with one of [compare]"

In [212]:
get_errs('DID_BASEFORM', 20)

["DID_BASEFORM: The verb 'can't' requires base form of this verb: 'compare' || Replace compares with one of [compare]",
 "DID_BASEFORM: The verb 'doesn't' requires base form of this verb: 'say' || Replace saying with one of [say]",
 "DID_BASEFORM: The verb 'can' requires the base form of the verb: 'be' || Replace is with one of [be]",
 "DID_BASEFORM: The verb 'will' requires the base form of the verb: 'drool' || Replace drooling with one of [drool]",
 "DID_BASEFORM: The verb 'can't' requires base form of this verb: 'repulse' || Replace repulsed with one of [repulse]",
 "DID_BASEFORM: The verb 'could' requires the base form of the verb: 'become' || Replace becomes with one of [become]",
 "DID_BASEFORM: The verb 'doesn't' requires base form of this verb: 'make' || Replace making with one of [make]",
 "DID_BASEFORM: The verb 'll' requires the base form of the verb: 'take' || Replace takes with one of [take]",
 "DID_BASEFORM: The verb 'can't' requires base form of this verb: 'behave' || Re

In [229]:
pd.options.display.max_colwidth = 500
err_data['Explanation'] = err_data['rule_id'].map(get_err)
err_data['Context'] = err_data['rule_id'].map(get_err_context)
err_data.head()

,freq_diff,orig_freq,pert_freq,freq_ratio,rule_id,freq_diff_perc,Explanation,Context
0,1315,22318,23633,1.058921,MORFOLOGIK_RULE_EN_US,0.422151,MORFOLOGIK_RULE_EN_US: Possible spelling mistake found || Replace davis with one of [Davis],davis is even enamored of her own creation th...
1,8,11008,11016,1.000727,UPPERCASE_SENTENCE_START,0.002568,UPPERCASE_SENTENCE_START: This sentence does not start with an uppercase letter || Replace davis with one of [Davis],davis is even enamored of her own creation th...
2,-426,7744,7318,0.944990,I_LOWERCASE,-0.136758,I_LOWERCASE: Did you mean 'I'? || Replace i with one of [I],"...een the first two cinema in the series, i can't compares friday after next to the..."
3,1,3399,3400,1.000294,COMMA_PARENTHESIS_WHITESPACE,0.000321,COMMA_PARENTHESIS_WHITESPACE: Don't put a space after the opening parenthesis || Replace ( with one of [(],rosenthal ( halloween iii ) sounds to recieve ignore...
4,0,1818,1818,1.000000,EN_QUOTES,0.000000,"EN_QUOTES: Use a smart opening quote here: '“'. || Replace "" with one of [“]","...anctimonious lloyd c douglas, (he wrote ""the robe""), and already filmed in 1935 w..."


In [221]:
err_data.sort_values(by=['freq_ratio','freq_diff'], ascending=False)

,freq_diff,orig_freq,pert_freq,freq_ratio,rule_id,freq_diff_perc,Explanation
30,36,2,38,19.000000,AGREEMENT_SENT_START,0.011557,AGREEMENT_SENT_START: You should probably use: 'are'. || Replace is with one of [are]
8,305,21,326,15.523810,DID_BASEFORM,0.097913,DID_BASEFORM: The verb 'can't' requires base form of this verb: 'compare' || Replace compares with one of [compare]
10,186,13,199,15.307692,NON3PRS_VERB,0.059711,NON3PRS_VERB: The pronoun 'i' must be used with a non-third-person form of a verb: 'surprise' || Replace surprises with one of [surprise]
7,306,24,330,13.750000,A_PLURAL,0.098234,"A_PLURAL: Don't use indefinite articles with plural words. Did you mean 'a grate', 'a gratis' or simply 'grates'? || Replace a grates with one of [a grate,a gratis,grates]"
27,44,4,48,12.000000,TO_NON_BASE,0.014125,"TO_NON_BASE: The verb after ""to"" should be in the base form: 'excuse'. || Replace excuses with one of [excuse]"
40,21,2,23,11.500000,PRP_PAST_PART,0.006742,PRP_PAST_PART: Possible grammatical error. You used a past participle without using any required verb ('be' or 'have'). Did you mean 'underwent'? || Replace undergone with one of [underwent]
64,10,1,11,11.000000,PRP_RB_NO_VB,0.003210,PRP_RB_NO_VB: Are you missing a verb? || Replace admittedly beenn with one of []
65,10,1,11,11.000000,COULDVE_IRREGULAR_VERB,0.003210,COULDVE_IRREGULAR_VERB: Did you mean 'i've had'? || Replace i've have with one of [i've had]
5,445,110,555,5.045455,EN_A_VS_AN,0.142857,"EN_A_VS_AN: Use 'an' instead of 'a' if the following word starts with a vowel sound, e.g. 'an article', 'an hour' || Replace a with one of [an]"
89,4,1,5,5.000000,NEEDS_FIXED,0.001284,"NEEDS_FIXED: ""wants constituted"" is only accepted in certain dialects. For something more widely acceptable, try 'constituting' or 'to be constituted'. || Replace constituted with one of [constituting,to be constituted]"


In [237]:
min_pert_freq = 25
min_pert_ratio = 5.0


explanatory_df = err_data \
    [(err_data['pert_freq'] > min_pert_freq) & (err_data['freq_ratio'] > min_pert_ratio)] \
    .sort_values(by=['freq_ratio','freq_diff'], ascending=False) \
    .reset_index() \
    .drop(['freq_ratio', 'freq_diff', 'index', 'freq_diff_perc'], axis=1) 

explanatory_df = explanatory_df[['rule_id', 'orig_freq', 'pert_freq', 'Explanation', 'Context']]
explanatory_df.head()

,rule_id,orig_freq,pert_freq,Explanation,Context
0,AGREEMENT_SENT_START,2,38,AGREEMENT_SENT_START: You should probably use: 'are'. || Replace is with one of [are],this films is too busiest beat all of its allotted ma...
1,DID_BASEFORM,21,326,DID_BASEFORM: The verb 'can't' requires base form of this verb: 'compare' || Replace compares with one of [compare],"...first two cinema in the series, i can't compares friday after next to them, but nothing ..."
2,NON3PRS_VERB,13,199,NON3PRS_VERB: The pronoun 'i' must be used with a non-third-person form of a verb: 'surprise' || Replace surprises with one of [surprise],...ved reached out hating the second one i surprises why they saw iike the same film to me
3,A_PLURAL,24,330,"A_PLURAL: Don't use indefinite articles with plural words. Did you mean 'a grate', 'a gratis' or simply 'grates'? || Replace a grates with one of [a grate,a gratis,grates]","a grates, lanky flick"
4,TO_NON_BASE,4,48,"TO_NON_BASE: The verb after ""to"" should be in the base form: 'excuse'. || Replace excuses with one of [excuse]",doesn't inbound close to excuses the hype that surrounded its debut at t...


In [246]:
latex = explanatory_df.to_latex()
for line in latex.split('\n'):
    print(line)

\begin{tabular}{llrrll}
\toprule
{} &               rule\_id &  orig\_freq &  pert\_freq &                                                                                                                                                                  Explanation &                                                                                         Context \\
\midrule
0 &  AGREEMENT\_SENT\_START &          2 &         38 &                                                                                        AGREEMENT\_SENT\_START: You should probably use: 'are'. || Replace is with one of [are] &                                        this films is too busiest beat all of its allotted ma... \\
1 &          DID\_BASEFORM &         21 &        326 &                                                          DID\_BASEFORM: The verb 'can't' requires base form of this verb: 'compare' || Replace compares with one of [compare] &  ...first two cinema in the series, i can't compares friday aft

In [249]:
df['run'].unique()

array(['movie_review_jin', 'yelp_polarity_jin', 'imdb_jin',
       'imdb_alzantot'], dtype=object)

In [250]:
len(df)

3115